In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from tqdm import tqdm

# 1. Cargar el Excel
df = pd.read_excel("/cont/news_Bancolombia.xlsx")

# 2. Definir modelo y tokenizer manualmente
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# 3. Inicializar el pipeline
sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# 4. Usar tqdm
tqdm.pandas()

# Función para aplicar el modelo y extraer resultados
def analizar_sentimiento(texto):
    if not isinstance(texto, str):
        return pd.Series([None, None])  # Si no es texto, devolvemos vacío
    resultado = sentiment_pipeline(texto[:512])[0]  # Truncamos a 512 tokens si es largo
    estrellas = int(resultado["label"][0])  # "3 stars" -> 3
    return pd.Series([resultado["label"], estrellas])

# 5. Aplicar el análisis a la columna "title"
df[["sentimiento_label", "sentimiento_estrellas"]] = df["title"].progress_apply(analizar_sentimiento)

# 6. Guardar el resultado en Excel
df.to_excel("noticias_banco_con_sentimiento.xlsx", index=False)

print("✅ Análisis de sentimiento completado y guardado en Excel.")